In [1]:
import random
import sys

import networkx as nx
import numpy as np
from dcmst.utils_cv import compute_n, is_cycle
from dcmst.utils import get_distance_table, get_distance

In [2]:
def get_permutation(random_key_sequence):
    key = np.array(random_key_sequence)
    return np.argsort(key)[::-1]

In [3]:
def decode(permutation, degree_constrained):
    n = compute_n(len(permutation))
    G = nx.complete_graph(n)
    # print("Full G:", G.edges)
    # print(G.edges[4])
    edges = list(G.edges)

    T = nx.empty_graph(n)
    # print(T.nodes)
    i = 0
    while len(T.edges) < n - 1:
        # T.add_edge(G.edges[i])
        j = permutation[i]
        # print(j)
        i += 1
        edge = edges[j]
        # print(edge)
        T.add_edge(*edge)
        nodes = [*edge]
        # print("Edge:", edge)
        # print("T:", T.edges)
        # print("Circle:", is_cycle(T))
        # print("Constrain:", T.degree(nodes[0]) > degree_constrained, T.degree(nodes[1]) > degree_constrained)
        if is_cycle(T):
            T.remove_edge(*edge)
        elif T.degree(nodes[0]) > degree_constrained or T.degree(nodes[1]) > degree_constrained:
            T.remove_edge(*edge)

    return T

In [4]:
def gen_key(n):
    l = int(n * (n - 1) / 2)
    return [random.random() for _ in range(l)]

In [5]:
def calculate_fitness(random_key_sequence, degree_constrained, distances_table):
    permutation = get_permutation(random_key_sequence)
    tree = decode(permutation, degree_constrained)
    edges = tree.edges
    # # Check if the degrees more than target degrees
    # if any(x > degree_constrained for x in degrees):
    #     return sys.maxsize

    cost = 0
    for edge in edges:
        cost = cost + get_distance(edge, distances_table)
    return cost

In [6]:
def crossover(parent1, parent2, crossover_rate=0.8):
    offspring1 = parent1[:]
    offspring2 = parent2[:]
    if random.random() < crossover_rate:
        crossover_point = random.randint(1, len(parent1) - 2)
        offspring1[crossover_point:], offspring2[crossover_point:] = offspring2[crossover_point:], offspring1[
                                                                                                   crossover_point:]
    return offspring1, offspring2


In [7]:


def mutate(individual, mutation_rate=0.1):
    if random.random() < mutation_rate:
        idx1, idx2 = random.sample(range(len(individual)), 2)
        individual[idx1], individual[idx2] = individual[idx2], individual[idx1]
    return individual


def mutate_pop(population, mutation_rate):
    new_pop = []
    for individual in population:
        if random.random() < mutation_rate:
            idx1, idx2 = random.sample(range(len(individual)), 2)
            individual[idx1], individual[idx2] = individual[idx2], individual[idx1]
            new_pop.append(individual)
    return new_pop

In [8]:
import heapq


def get_new_pop(population, population_size, degree_constrained, distances_table):
    fitness_values = [calculate_fitness(individual, degree_constrained, distances_table) for
                      individual in population]
    sorted_fitness = sorted(fitness_values)
    index_dict = {val: idx for idx, val in enumerate(fitness_values)}
    # Lấy danh sách index của population đã được sắp xếp theo fitness
    pop_index = [index_dict[val] for val in sorted_fitness]
    # Lấy index của [population_size] phần tử đầu (có cost nhỏ nhất)
    new_pop_index = pop_index[:population_size]
    new_pop = [population[i] for i in new_pop_index]
    return new_pop


def create_new_population(population, fitness, pop_size):
    # create a list of individuals with their fitnesses
    individuals_fitness = [(population[i], fitness[i]) for i in range(len(population))]

    # create a new population with the n/2 best individuals
    best_individuals = heapq.nsmallest(pop_size // 2, individuals_fitness, key=lambda x: x[1])
    new_pop = [individual[0] for individual in best_individuals]

    # create a list of remaining individuals
    remaining_individuals = individuals_fitness[pop_size // 2:]
    # add n/2 individuals chosen randomly with the criterion of the best fitness
    for _ in range(pop_size // 2):
        # print("remaining", remaining_individuals)
        # randomly select two individuals
        ind_indices = np.random.choice(len(remaining_individuals), size=2, replace=False)
        #         print(ind_indices)
        ind1, ind2 = remaining_individuals[ind_indices[0]], remaining_individuals[ind_indices[1]]
        #         print(ind1, ind2)
        # add the individual with the highest fitness to the new population
        if ind1[1] > ind2[1]:
            new_pop.append(ind1[0])
            remaining_individuals.remove(ind1)
        else:
            new_pop.append(ind2[0])
            remaining_individuals.remove(ind2)

    return new_pop

In [9]:

def run_ga(n, degree_constrained, distances_table, population_size=50, crossover_rate=0.8, mutation_rate=0.1,
           max_generations=50):
    population = [gen_key(n) for _ in range(population_size)]
    # print(population)
    for generation in range(max_generations):
        # print(population)
        # Tính một mảng fitness value của population
        fitness_values = [calculate_fitness(cv_sequence, degree_constrained, distances_table) for
                          cv_sequence in population]
        # print(fitness_values)
        new_population = []
        while len(new_population) < population_size:
            i, j = np.random.choice(range(population_size), size=2, replace=False,
                                    p=np.array(fitness_values) / sum(fitness_values))
            parent1 = population[i]
            parent2 = population[j]
            offspring1, offspring2 = crossover(parent1, parent2)
            new_population.append(offspring1)
            new_population.append(offspring2)
        for i in range(len(new_population)):
            mutate(new_population[i])
        new_fitness = [calculate_fitness(prufer_sequence, degree_constrained, distances_table) for
                          prufer_sequence in new_population]
        fitness = fitness_values + new_fitness

        population = population + new_population
        population = create_new_population(population, fitness, population_size)
        # population = get_new_pop(population, population_size, degree_constrained, distances_table)

    return population


In [10]:
degree_constrained = 2
n = 5
distances_table = get_distance_table("data/5_wi29.csv")

population = run_ga(n, degree_constrained, distances_table)
print(population[0])
print(calculate_fitness(population[0], degree_constrained, distances_table))

[0.9827479083082298, 0.09457848200699359, 0.038412987173663926, 0.2550253531083585, 0.5405597922371816, 0.5882802806259226, 0.9594093750254677, 0.2555215909487325, 0.18994478997945385, 0.2573316066242348]
4277.163670389418


In [11]:
T = decode(get_permutation(population[0]), degree_constrained)
print(T.edges)
print(nx.is_tree(T))

[(0, 1), (1, 4), (2, 3), (3, 4)]
True


In [12]:

# degree_constrained = 3
# n = 99
# distances_table = get_distance_table("data/99_fi10k.csv")
#
# population = run_ga(n, degree_constrained, distances_table)
# print(population[0])
# print(calculate_fitness(population[0], degree_constrained, distances_table))
#
# T = decode(get_permutation(population[0]), degree_constrained)
# print(T.edges)
# print(nx.is_tree(T))

In [13]:
import csv


import csv


def write_result(results, filename):
    path_result = "result"
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['STT', 'Best Cost', 'Best Tree', 'Time', 'Is Tree?'])

        # Ghi từng dòng kết quả
        for result in results:
            writer.writerow(result)

In [14]:
import os
import time

degree_constrained = 3
n = 60
path = "data"
list_file = [file_name for file_name in os.listdir(path) if file_name.endswith(".csv")]
print(list_file)
for file in list_file:
    print(file)
    results = []
    path_to_data = os.path.join(path, file)
    dis_tab = get_distance_table(path_to_data)
    path_to_result = path_to_data.replace("data", "result_nk")
    path_to_result = path_to_result.replace("\\", "/")
    # print(path_to_result)
    for i in range(10):
        print("Loop: ", i + 1, "...")
        start_time = time.time()
        population = run_ga(n, degree_constrained, dis_tab)
        best_solution = population[0]
        best_cost = calculate_fitness(best_solution, n, dis_tab)
        permutation = get_permutation(population[0])
        best_tree = decode(permutation, degree_constrained).edges()
        end_time = time.time()
        G = nx.from_edgelist(best_tree)
        isTree = nx.is_tree(G)
        elapsed_time = end_time - start_time
        row = (i + 1, best_cost, best_tree, elapsed_time, isTree)
        results.append(row)
    print("Result 10 times", results)

    # print(path_to_result)
    write_result(results, path_to_result)

['10_15_fi10k.csv', '11_15_fi10k.csv', '12_15_fi10k.csv', '13_15_fi10k.csv', '14_15_fi10k.csv', '15_15_fi10k.csv', '16_15_fi10k.csv', '17_15_fi10k.csv', '18_15_fi10k.csv', '19_15_fi10k.csv', '1_15_fi10k.csv', '20_15_fi10k.csv', '21_15_fi10k.csv', '22_15_fi10k.csv', '23_15_fi10k.csv', '24_15_fi10k.csv', '25_15_fi10k.csv', '26_15_fi10k.csv', '27_15_fi10k.csv', '28_15_fi10k.csv', '29_15_fi10k.csv', '2_15_fi10k.csv', '30_15_fi10k.csv', '3_15_fi10k.csv', '4_15_fi10k.csv', '5_15_fi10k.csv', '6_15_fi10k.csv', '7_15_fi10k.csv', '8_15_fi10k.csv', '9_15_fi10k.csv']
10_15_fi10k.csv
Loop:  1 ...
Loop:  2 ...
Loop:  3 ...
Loop:  4 ...
Loop:  5 ...
Loop:  6 ...
Loop:  7 ...
Loop:  8 ...
Loop:  9 ...
Loop:  10 ...
Result 10 times [(1, [0.8918347006967837, 0.2045095107747813, 0.7669708799363558, 0.4594463587949319, 0.4905272742231457, 0.01595125113123952, 0.8495809933485424, 0.24897466763818077, 0.6011215891431139, 0.468032824202509, 0.1179099243853372, 0.9177030781489697, 0.8633891377158643, 0.940183